# Laboratorio 9

In [1]:
from zipfile import ZipFile
from pathlib import Path
import pandas as pd

Path.cwd()

WindowsPath('d:/Documenti/Git_repositories/Data_science_lab/data-science-lab-1/MyRepo/Lab9')

In [2]:
with ZipFile('NYC_Airbnb.zip') as zipped_file:
    zipped_file.extractall(Path.cwd())

1. Load the dataset from the root folder

In [3]:
df_dev = pd.read_csv(Path.cwd()/'NYC_Airbnb'/'development.csv', index_col=0)
df_eval = pd.read_csv(Path.cwd()/'NYC_Airbnb'/'evaluation.csv', index_col=0)

In [4]:
df_dev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39116 entries, 12783632 to 15310580
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   name                            39103 non-null  object 
 1   host_id                         39116 non-null  int64  
 2   host_name                       39097 non-null  object 
 3   neighbourhood_group             39116 non-null  object 
 4   neighbourhood                   39116 non-null  object 
 5   latitude                        39116 non-null  float64
 6   longitude                       39116 non-null  float64
 7   room_type                       39116 non-null  object 
 8   price                           39116 non-null  int64  
 9   minimum_nights                  39116 non-null  int64  
 10  number_of_reviews               39116 non-null  int64  
 11  last_review                     31075 non-null  object 
 12  reviews_per_month     

In [5]:
df_dev.head()

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
id,,,,,,,,,,,,,,,
12783632,NYC Mini Hotel,57230304,Imanuelly,Queens,Elmhurst,40.74037,-73.88610,Private room,75,1,2,2019-05-26,0.92,3,351
3463385,Gorgeous room in Manhattan,10698270,Evgenia,Manhattan,Upper East Side,40.76717,-73.95532,Private room,95,1,202,2019-05-27,3.31,2,263
17572926,Great 1 Bedroom on Upper East,36578169,James,Manhattan,Upper East Side,40.77984,-73.94725,Entire home/apt,130,2,0,NaN,NaN,1,0
33913644,"Modern and bright 2Bed 2Bath Bushwick, Brooklyn",50981314,Ofier,Brooklyn,Bushwick,40.70205,-73.91338,Entire home/apt,150,2,4,2019-07-07,1.64,1,89
9405895,Stylish and zen Brooklyn retreat,48775347,Mathieu,Brooklyn,Fort Greene,40.68914,-73.97853,Entire home/apt,325,3,16,2019-04-20,0.42,1,103


In [6]:
df = pd.concat([df_dev,df_eval],axis=0)

In [7]:
df.isna().any()

name                               True
host_id                           False
host_name                          True
neighbourhood_group               False
neighbourhood                     False
latitude                          False
longitude                         False
room_type                         False
price                              True
minimum_nights                    False
number_of_reviews                 False
last_review                        True
reviews_per_month                  True
calculated_host_listings_count    False
availability_365                  False
dtype: bool

In [8]:
df['reviews_per_month'].fillna(0,inplace=True)

Dummy:  
'neighbourhood_group', 'neighbourhood','room_type'  
Discard:  
'host_id','name','host_name','last_review'

In [9]:
df_1h = pd.get_dummies(df,columns=['neighbourhood_group', 'neighbourhood','room_type'])
df_dropped = df_1h.drop(['host_id','name','host_name','last_review'], axis=1)

In [10]:
from sklearn.model_selection import train_test_split

train_and_val_mask = df_dropped['price'].notna()

feature_names = df_dropped.drop('price',axis=1).columns

X = df_dropped.loc[:,list(feature_names)].values
y = df_dropped['price'].values

X_train_and_val = X[train_and_val_mask]
y_train_and_val = y[train_and_val_mask]
X_test = X[~train_and_val_mask]
y_test = y[~train_and_val_mask]

X_train, X_valid, y_train, y_valid= train_test_split(X_train_and_val, y_train_and_val, shuffle=True, random_state=42)

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

reg = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-2)
reg.fit(X_train,y_train)
y_pred = reg.predict(X_valid)
r2_score(y_valid,y_pred)

0.07128877883880957

In [12]:
reg_feature_importances_1 = pd.Series(reg.feature_importances_, index=feature_names, name='importances').sort_values(ascending=False)
reg_feature_importances_1

longitude                       2.011533e-01
latitude                        2.007119e-01
availability_365                9.798847e-02
minimum_nights                  8.811410e-02
reviews_per_month               6.634125e-02
                                    ...     
neighbourhood_New Brighton      1.698312e-09
neighbourhood_Richmondtown      0.000000e+00
neighbourhood_Neponsit          0.000000e+00
neighbourhood_Woodrow           0.000000e+00
neighbourhood_Fort Wadsworth    0.000000e+00
Name: importances, Length: 236, dtype: float64

Dummy:  
'neighbourhood_group', 'neighbourhood','room_type'  
Discard:  
'host_id','name','host_name','last_review'

In [13]:
df_1h = pd.get_dummies(df, columns=['room_type'])
df_dropped = df_1h.drop(['neighbourhood_group', 'neighbourhood','host_id','name','host_name','last_review'],axis=1)

train_and_val_mask = df_dropped['price'].notna()

feature_names = df_dropped.drop('price',axis=1).columns

X = df_dropped.drop('price',axis=1).values
y = df_dropped['price'].values

X_train_and_val = X[train_and_val_mask]
y_train_and_val = y[train_and_val_mask]
X_test = X[~train_and_val_mask]
y_test = y[~train_and_val_mask]

X_train, X_valid, y_train, y_valid= train_test_split(X_train_and_val, y_train_and_val, shuffle=True, random_state=42)

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

reg = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-2)
reg.fit(X_train,y_train)
y_pred = reg.predict(X_valid)
r2_score(y_valid,y_pred)

0.10485905409912633

In [15]:
reg_feature_importances_1 = pd.Series(reg.feature_importances_, index=feature_names, name='importances').sort_values(ascending=False)
reg_feature_importances_1

longitude                         0.310636
latitude                          0.243221
availability_365                  0.102062
minimum_nights                    0.094534
reviews_per_month                 0.070677
room_type_Entire home/apt         0.063075
calculated_host_listings_count    0.060979
number_of_reviews                 0.053690
room_type_Shared room             0.000705
room_type_Private room            0.000421
Name: importances, dtype: float64

Including 'id' and 'host_id' in the analysis

In [16]:
df_1h = pd.get_dummies(df, columns=['room_type'])
df_1h['id'] = df_1h.index
df_dropped = df_1h.drop(['neighbourhood_group', 'neighbourhood','name','host_name','last_review'],axis=1)

train_and_val_mask = df_dropped['price'].notna()

feature_names = df_dropped.drop('price',axis=1).columns

X = df_dropped.drop('price',axis=1).values
y = df_dropped['price'].values

X_train_and_val = X[train_and_val_mask]
y_train_and_val = y[train_and_val_mask]
X_test = X[~train_and_val_mask]
y_test = y[~train_and_val_mask]

X_train, X_valid, y_train, y_valid= train_test_split(X_train_and_val, y_train_and_val, shuffle=True, random_state=42)

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

reg = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-2)
reg.fit(X_train,y_train)
y_pred = reg.predict(X_valid)
r2_score(y_valid,y_pred)

0.13278619065870456

In [18]:
reg_feature_importances_1 = pd.Series(reg.feature_importances_, index=feature_names, name='importances').sort_values(ascending=False)
reg_feature_importances_1

longitude                         0.203147
host_id                           0.176144
id                                0.146732
latitude                          0.130673
minimum_nights                    0.078468
availability_365                  0.077420
room_type_Entire home/apt         0.063075
reviews_per_month                 0.043341
calculated_host_listings_count    0.043281
number_of_reviews                 0.037004
room_type_Private room            0.000435
room_type_Shared room             0.000280
Name: importances, dtype: float64

### Exploration of the post titles

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    stop_words="english", 
    #binary=False, 
    binary=True, 
    use_idf=False, 
    norm=False
)

wpm = vectorizer.fit_transform(df_1h["name"].fillna(""))

print(wpm)

  (0, 5141)	1.0
  (0, 4827)	1.0
  (0, 3901)	1.0
  (1, 3587)	1.0
  (1, 5951)	1.0
  (1, 4627)	1.0
  (2, 3630)	1.0
  (2, 1709)	1.0
  (2, 7092)	1.0
  (2, 3002)	1.0
  (3, 4879)	1.0
  (3, 1961)	1.0
  (3, 448)	1.0
  (3, 434)	1.0
  (3, 2069)	1.0
  (3, 1987)	1.0
  (4, 1987)	1.0
  (4, 6586)	1.0
  (4, 7534)	1.0
  (4, 5874)	1.0
  (5, 1684)	1.0
  (5, 6386)	1.0
  (5, 3658)	1.0
  (5, 3364)	1.0
  (5, 1404)	1.0
  :	:
  (48889, 5310)	1.0
  (48890, 3257)	1.0
  (48890, 7313)	1.0
  (48891, 7370)	1.0
  (48891, 6559)	1.0
  (48891, 5061)	1.0
  (48891, 7515)	1.0
  (48891, 1442)	1.0
  (48891, 5734)	1.0
  (48892, 5141)	1.0
  (48892, 1709)	1.0
  (48892, 2211)	1.0
  (48892, 5319)	1.0
  (48892, 6574)	1.0
  (48892, 7250)	1.0
  (48893, 1709)	1.0
  (48893, 2252)	1.0
  (48893, 5833)	1.0
  (48893, 5065)	1.0
  (48893, 5996)	1.0
  (48894, 5951)	1.0
  (48894, 3002)	1.0
  (48894, 2069)	1.0
  (48894, 7370)	1.0
  (48894, 1253)	1.0


In [20]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# corpus = [
#     'This is the first document.',
#     'This document is the second document.',
#     'And this is the third one.',
#     'Is this the first document?',
# ]
# vectorizer = TfidfVectorizer(stop_words='english')
# X = vectorizer.fit_transform(corpus)
# vectorizer.get_feature_names_out()


# print(vectorizer.get_feature_names_out())
# print(X.toarray())

In [21]:
import numpy as np
N = 150

word_freq = pd.Series(np.asarray(wpm.sum(axis=0)).squeeze(),index=vectorizer.get_feature_names_out()).sort_values(ascending=False)[:N]

word_freq

room         10213.0
bedroom       8145.0
private       7275.0
apartment     6758.0
cozy          5093.0
              ...   
3br            284.0
greene         280.0
bedrooms       279.0
patio          277.0
time           276.0
Length: 150, dtype: float64

In [22]:
word_ind = [w in word_freq.index for w in vectorizer.get_feature_names_out()]
word_ind
# a = np.array(vectorizer.get_feature_names_out())[word_ind]
# b = np.sort(np.array(word_freq.index))
# (a==b).all()
# vectorizer.get_feature_names_out()[word_ind]

words_df = pd.DataFrame(
    data = wpm[:,word_ind].toarray(),
    columns = vectorizer.get_feature_names_out()[word_ind],
    index = df_1h.index
).add_prefix('word_')

In [23]:
df_1h = pd.get_dummies(df,columns=['room_type'])

df_1h = df_1h.join(words_df)

df_dropped = df_1h.drop(columns=["neighbourhood_group","neighbourhood", "name","host_name", "last_review"])

df_dropped["id"] = df_dropped.index
train_valid_mask = df_dropped["price"].notna()
feature_names = df_dropped[train_valid_mask].drop(columns=["price"]).columns
X = df_dropped.drop(columns=["price"]).values
y = df_dropped["price"].values
X_train_valid = X[train_valid_mask]
y_train_valid = y[train_valid_mask]
X_test = X[~train_valid_mask]
y_test = y[~train_valid_mask]
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid,y_train_valid, shuffle=True, random_state=42)
reg = RandomForestRegressor(100, random_state=42, n_jobs=-2)
reg.fit(X_train, y_train)
r2_score(y_valid, reg.predict(X_valid))

0.16344001987598977

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    "n_estimators": [10,20,30],
    "criterion": ["mse", "mae"],
    "max_features": ["auto", "sqrt", "log2"],
    "random_state": [42],
    "n_jobs": [-1],
}

gs = GridSearchCV(RandomForestRegressor(), param_grid, scoring="r2", n_jobs=-1,cv=5)
gs.fit(X_train_valid, y_train_valid)

gs.best_score_
